In [ ]:
# This reads in files of period info, epochs-of-max, and spectra BJDs to 
# find the phases as the time the spectra were taken 

# Created 2022 May 23 by E.S.

In [1]:
import pandas as pd
import numpy as np

In [4]:
df_spectra_epochs = pd.read_csv("./data/spec_bjds.csv")
df_stellar_periods = pd.read_csv("./data/output_periods_all_stars.csv")
df_epochs_of_max = pd.read_csv("./data/bjds_epochs_of_max.csv")

In [57]:
# make column in each dataframe that will be used to match by star

df_spectra_epochs["star_match"] = df_spectra_epochs["file"]
df_spectra_epochs["star_match"] = df_spectra_epochs["star_match"].str.replace("__"," ") # underscore of 2
df_spectra_epochs["star_match"] = df_spectra_epochs["star_match"].str.replace("_"," ") # underscore of 1
df_spectra_epochs["star_match"] = df_spectra_epochs["star_match"].apply(lambda row: row.rsplit(' ',1)[0])

df_epochs_of_max["star_match"] = df_epochs_of_max["star"].str.replace("_"," ")

df_stellar_periods["star_match"] = df_stellar_periods["star"]

In [59]:
# remove some ambiguities

df_epochs_of_max["photo_bjd"] = df_epochs_of_max["bjd"]
df_spectra_epochs["spec_bjd"] = df_spectra_epochs["bjd"]

In [61]:
# combine epochs-of-max and spectral epochs
result = pd.merge(df_spectra_epochs,
                 df_epochs_of_max[['star_match','photo_bjd']],
                 on='star_match')

# combine with periods
result2 = pd.merge(result,
                 df_stellar_periods[['star_match','T_final','err_tot']],
                 on='star_match')

In [114]:
result2["baseline_time"].where(idx_pos)

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
5            NaN
6            NaN
7            NaN
8            NaN
9            NaN
10           NaN
11           NaN
12           NaN
13           NaN
14           NaN
15           NaN
16           NaN
17           NaN
18           NaN
19           NaN
20           NaN
21           NaN
22           NaN
23           NaN
24           NaN
25           NaN
26           NaN
27           NaN
28           NaN
29           NaN
         ...    
94           NaN
95           NaN
96           NaN
97           NaN
98           NaN
99           NaN
100          NaN
101          NaN
102          NaN
103          NaN
104          NaN
105          NaN
106          NaN
107          NaN
108          NaN
109          NaN
110          NaN
111          NaN
112          NaN
113          NaN
114          NaN
115          NaN
116          NaN
117          NaN
118    10.201427
119     9.170285
120     9.040187
121     9.0845

In [121]:
# find phases

result2["phasemod"] = np.nan
result2["baseline_time"] = np.subtract(result2["spec_bjd"],result2["photo_bjd"])

# for cases where spectra were taken after photometry
idx_pos = (result2["baseline_time"] > 0)
result2["phasemod"].loc[idx_pos] = np.mod(result2["baseline_time"].where(idx_pos),result2["T_final"].where(idx_pos))

# vice versa
idx_neg = (result2["baseline_time"] < 0)
result2["phasemod"].loc[idx_neg] = np.subtract(1.,np.mod(result2["baseline_time"].where(idx_neg),result2["T_final"].where(idx_neg)))

In [123]:
result2

,Unnamed: 0,file,ut_epoch,ra,dec,jd,bjd,star_match,spec_bjd,photo_bjd,T_final,err_tot,baseline_time,phasemod
0,0,BH_Peg_03.fits,2013-07-24 11:12:07.52,22:53:49.60,+15:47:25.50,2.456498e+06,2.456498e+06,BH Peg,2.456498e+06,2.456564e+06,0.640993,0.000007,-65.660552,0.638273
1,29,BH_Peg_02.fits,2013-07-24 10:25:10.29,22:53:47.90,+15:47:21.10,2.456498e+06,2.456498e+06,BH Peg,2.456498e+06,2.456564e+06,0.640993,0.000007,-65.693160,0.670881
2,99,BH_Peg_01.fits,2013-07-24 09:36:23.75,22:53:47.40,+15:46:56.50,2.456498e+06,2.456498e+06,BH Peg,2.456498e+06,2.456564e+06,0.640993,0.000007,-65.727034,0.704755
3,1,RW_Ari_02.fits,2012-12-27 03:07:05.85,02:16:48.20,+17:30:31.60,2.456289e+06,2.456289e+06,RW Ari,2.456289e+06,2.456314e+06,0.354434,0.000016,-24.921681,0.756841
4,30,RW_Ari_03.fits,2012-12-27 04:04:11.28,02:16:49.40,+17:30:52.70,2.456289e+06,2.456289e+06,RW Ari,2.456289e+06,2.456314e+06,0.354434,0.000016,-24.882038,0.717198
5,42,RW_Ari_04.fits,2012-12-28 02:01:29.47,02:16:46.80,+17:30:55.40,2.456290e+06,2.456290e+06,RW Ari,2.456290e+06,2.456314e+06,0.354434,0.000016,-23.967324,0.865786
6,58,RW_Ari_05.fits,2012-12-28 03:15:11.67,02:16:48.10,+17:30:55.40,2.456290e+06,2.456290e+06,RW Ari,2.456290e+06,2.456314e+06,0.354434,0.000016,-23.916145,0.814608
7,115,RW_Ari_01.fits,2012-12-27 02:09:44.74,02:16:47.10,+17:30:57.10,2.456289e+06,2.456289e+06,RW Ari,2.456289e+06,2.456314e+06,0.354434,0.000016,-24.961506,0.796666
8,2,AV_Peg_08.fits,2013-07-26 08:03:17.91,21:52:45.40,+22:34:05.70,2.456500e+06,2.456500e+06,AV Peg,2.456500e+06,2.456594e+06,0.390382,0.000007,-93.748673,0.666688
9,9,AV_Peg_04.fits,2013-07-25 07:39:22.39,21:52:44.90,+22:33:31.40,2.456499e+06,2.456499e+06,AV Peg,2.456499e+06,2.456594e+06,0.390382,0.000007,-94.765336,0.902588


In [124]:
result3 = result2.sort_values(by="file").reset_index(drop=True)

In [140]:
for t in range(0,len(result3)):
    print(result3["file"].loc[t])
    print(result3["spec_bjd"].loc[t])
    print("----")

AR_Per_01.fits
2456282.6822751886
----
AR_Per_02.fits
2456282.691126706
----
AR_Per_03.fits
2456282.761346463
----
AR_Per_04.fits
2456282.803432178
----
AR_Per_05.fits
2456282.819708069
----
AR_Per_06.fits
2456282.8327525686
----
AR_Per_07.fits
2456282.848764036
----
AR_Per_08.fits
2456283.6977288728
----
AR_Per_09.fits
2456283.7535911626
----
AR_Per_10.fits
2456283.8249799497
----
AR_Per_11.fits
2456285.6469672243
----
AR_Per_12.fits
2456285.7601578347
----
AR_Per_13.fits
2456285.77339128
----
AR_Per_14.fits
2456285.7892347286
----
AR_Per_15.fits
2456286.6835516985
----
AR_Per_16.fits
2456289.7496305034
----
AV_Peg_01.fits
2456497.86998138
----
AV_Peg_02.fits
2456497.9223477743
----
AV_Peg_03.fits
2456497.954415798
----
AV_Peg_04.fits
2456498.8237113687
----
AV_Peg_05.fits
2456498.8491927776
----
AV_Peg_06.fits
2456498.873874892
----
AV_Peg_07.fits
2456498.9331899458
----
AV_Peg_08.fits
2456499.840374843
----
AV_Peg_09.fits
2456499.8649172536
----
BH_Peg_01.fits
2456497.9045969616
---

In [134]:
i_test = 79
print(result3["spec_bjd"][i_test])
print(result2["phasemod"][i_test])
print(result2["baseline_time"][i_test])
print(result2["star_match"][i_test])

2456289.7703986457
0.7462384943914593
-323.07476551039144
TW Lyn


2456282.6822751886

0.6382727105199361

'TV Lyn'

In [3]:
df_phases["epoch_round"] = np.round(df_phases["Dates"]-2400000,5)

In [4]:
df_file_names["epoch_round"] = np.round(df_file_names["bjd"],5)

In [5]:
df = pd.merge(df_phases, df_file_names, how = "right", on = ["epoch_round"])

In [6]:
df2 = df[["#Name","filenames","Period","bjd","Phases"]].copy().drop_duplicates()

In [7]:
# replace '.fits' with '.dat'

df2["filenames"] = df2["filenames"].str.replace('.fits','.dat')

In [8]:
print(df2.to_string())

                 #Name        filenames    Period           bjd    Phases
0       AR_Per_E.cur_0    AR_Per_16.dat  0.425551  56289.749621  0.274712
1      AR_Per_E.cur_31    AR_Per_16.dat  0.425551  56289.749621  0.274712
2       AR_Per_E.cur_0  AR_Per_16.c.dat  0.425551  56289.749621  0.274712
3      AR_Per_E.cur_31  AR_Per_16.c.dat  0.425551  56289.749621  0.274712
4      AR_Per_E.cur_10  AR_Per_05.c.dat  0.425551  56282.819699  0.990115
5      AR_Per_E.cur_21  AR_Per_05.c.dat  0.425551  56282.819699  0.990115
6      AR_Per_E.cur_10    AR_Per_05.dat  0.425551  56282.819699  0.990115
7      AR_Per_E.cur_21    AR_Per_05.dat  0.425551  56282.819699  0.990115
8      AR_Per_E.cur_11  AR_Per_09.c.dat  0.425551  56283.753583  0.184644
9      AR_Per_E.cur_22  AR_Per_09.c.dat  0.425551  56283.753583  0.184644
10     AR_Per_E.cur_11    AR_Per_09.dat  0.425551  56283.753583  0.184644
11     AR_Per_E.cur_22    AR_Per_09.dat  0.425551  56283.753583  0.184644
12     AR_Per_E.cur_12  AR_Per_07.c.da

In [9]:
# Note that there are effectively repeated lines in this dataframe, but the phases are accurate matches

df2.to_csv("./data/junk1.csv",index=False)

In [10]:
# Option to read in a file name list that is for input to the pipeline, and matching phases

df_input_list = pd.read_csv("/Users/bandari/Documents/git.repos/rrlyrae_metallicity/src/mcd_beta_phases_ascii_files_all.list")
df_phase_info = pd.read_csv("./data/matched_phases_20211117.csv")
df_phase_info["orig_spec_file_name"] = df_phase_info["filenames"] # add col to enable match

In [19]:
df_input_list_updated = pd.merge(df_input_list, df_phase_info, how = "left", on = ["orig_spec_file_name"])
df_input_list_updated["phase"] = df_input_list_updated["Phases"]
df_input_list_updated = df_input_list_updated[["orig_spec_file_name","subtype","phase","feh","err_feh"]]
df_input_list_updated = df_input_list_updated.drop_duplicates()

In [20]:
df_input_list_updated

,orig_spec_file_name,subtype,phase,feh,err_feh
0,AR_Per_01.dat,NaN,0.667164,NaN,NaN
2,AR_Per_02.dat,NaN,0.687964,NaN,NaN
4,AR_Per_03.dat,NaN,0.852972,NaN,NaN
6,AR_Per_04.dat,NaN,0.951869,NaN,NaN
8,AR_Per_05.dat,NaN,0.990115,NaN,NaN
...,...,...,...,...,...
265,X_Ari__06.dat,NaN,NaN,NaN,NaN
266,X_Ari__07.dat,NaN,NaN,NaN,NaN
267,X_Ari__08.dat,NaN,NaN,NaN,NaN
268,X_Ari__09.dat,NaN,NaN,NaN,NaN


In [21]:
df_input_list_updated.to_csv("./data/junk.csv", index=False)